# ICS5110: Applied Machine Learning

Peter Galea St. John: 288098m\
Christian A. Vella


In [76]:
# Import some necessary libraries

import pandas as pd
import math
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import graphviz
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

######-ANN
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder


######-RF
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# Data Preparation

In [77]:
 with open('Cleaned-Data.csv') as csv_file:
    data = list(csv.reader(csv_file, delimiter='\t'))
    r = len(data)

In [78]:
df = pd.read_csv('Cleaned-Data.csv',na_values="?" )

In [79]:
df.drop(columns="Country")

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Gender_Female,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes
0,1,1,1,1,1,0,1,1,1,1,...,0,1,0,1,0,0,0,0,0,1
1,1,1,1,1,1,0,1,1,1,1,...,0,1,0,1,0,0,0,0,1,0
2,1,1,1,1,1,0,1,1,1,1,...,0,1,0,1,0,0,0,1,0,0
3,1,1,1,1,1,0,1,1,1,1,...,0,1,0,0,1,0,0,0,0,1
4,1,1,1,1,1,0,1,1,1,1,...,0,1,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316795,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
316796,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
316797,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
316798,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0


In [80]:
df.head()

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country
0,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,0,0,1,China
1,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,0,1,0,China
2,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,1,0,0,China
3,1,1,1,1,1,0,1,1,1,1,...,1,0,0,1,0,0,0,0,1,China
4,1,1,1,1,1,0,1,1,1,1,...,1,0,0,1,0,0,0,1,0,China


In [81]:
severity_columns = df.filter(like='Severity_').columns

df['Severity_None'].replace({1:'0',0:'No'},inplace =True)
df['Severity_Mild'].replace({1:'1',0:'No'},inplace =True)
df['Severity_Moderate'].replace({1:'2',0:'No'},inplace =True)
df['Severity_Severe'].replace({1:'3',0:'No'},inplace =True)

df['Condition'] = df[severity_columns].values.tolist()

df.drop(columns=['Severity_None','Severity_Mild','Severity_Moderate','Severity_Severe'])

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Age_25-59,Age_60+,Gender_Female,Gender_Male,Gender_Transgender,Contact_Dont-Know,Contact_No,Contact_Yes,Country,Condition
0,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,0,0,1,China,"[1, No, No, No]"
1,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,0,1,0,China,"[1, No, No, No]"
2,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,1,0,0,China,"[1, No, No, No]"
3,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,0,0,1,China,"[No, 2, No, No]"
4,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,0,1,0,China,"[No, 2, No, No]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316795,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,1,0,Other,"[No, No, No, 3]"
316796,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,1,0,0,Other,"[No, No, No, 3]"
316797,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,1,Other,"[No, No, 0, No]"
316798,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,1,0,Other,"[No, No, 0, No]"


In [82]:
def removing(list1):
    list1 = set(list1) 
    list1.discard("No")
    a = ''.join(list1)
    return a

In [83]:
df['Condition'] = df['Condition'].apply(removing)

In [84]:
df['Severity_None'].replace({'1': 1,'No':0},inplace =True)
df['Severity_Mild'].replace({'1':1,'No':0},inplace =True)
df['Severity_Moderate'].replace({1:'2',0:'No'},inplace =True)
df['Severity_Severe'].replace({1:'3',0:'No'},inplace =True)

In [85]:
df.head()

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country,Condition
0,1,1,1,1,1,0,1,1,1,1,...,0,1,No,0,No,0,0,1,China,1
1,1,1,1,1,1,0,1,1,1,1,...,0,1,No,0,No,0,1,0,China,1
2,1,1,1,1,1,0,1,1,1,1,...,0,1,No,0,No,1,0,0,China,1
3,1,1,1,1,1,0,1,1,1,1,...,0,0,2,0,No,0,0,1,China,2
4,1,1,1,1,1,0,1,1,1,1,...,0,0,2,0,No,0,1,0,China,2


In [86]:
headers= df.columns.tolist()

In [87]:
df = df.sample(n=30000, replace=True,random_state=1)

In [88]:
X= pd.DataFrame.to_numpy(df.drop(['Condition','Country'],axis=1))
y= pd.DataFrame.to_numpy(df['Condition'])

In [89]:
X= np.delete(X,19,1)
X= np.delete(X,19,1)
X= np.delete(X,19,1)
X= np.delete(X,19,1)

In [90]:
print(headers)
headers.remove('Severity_Mild')
headers.remove('Severity_Moderate')
headers.remove('Severity_None')
headers.remove('Severity_Severe')
headers.remove('Condition')
headers.remove('Country')
print(headers)

['Fever', 'Tiredness', 'Dry-Cough', 'Difficulty-in-Breathing', 'Sore-Throat', 'None_Sympton', 'Pains', 'Nasal-Congestion', 'Runny-Nose', 'Diarrhea', 'None_Experiencing', 'Age_0-9', 'Age_10-19', 'Age_20-24', 'Age_25-59', 'Age_60+', 'Gender_Female', 'Gender_Male', 'Gender_Transgender', 'Severity_Mild', 'Severity_Moderate', 'Severity_None', 'Severity_Severe', 'Contact_Dont-Know', 'Contact_No', 'Contact_Yes', 'Country', 'Condition']
['Fever', 'Tiredness', 'Dry-Cough', 'Difficulty-in-Breathing', 'Sore-Throat', 'None_Sympton', 'Pains', 'Nasal-Congestion', 'Runny-Nose', 'Diarrhea', 'None_Experiencing', 'Age_0-9', 'Age_10-19', 'Age_20-24', 'Age_25-59', 'Age_60+', 'Gender_Female', 'Gender_Male', 'Gender_Transgender', 'Contact_Dont-Know', 'Contact_No', 'Contact_Yes']


In [91]:
y.shape

(30000,)

____

## Feature Selection: Chi Square

In [92]:
def get_max(x):
    max_val = 0
    for i in x:
        if i>max_val:
            max_val = i
    return max_val

In [93]:
def get_min(x):
    min_val = 99
    for i in x:
        if i<min_val:
            min_val = i
    return min_val

In [94]:
y = y.astype(np.int)

### Contingency Table

In [95]:
contingency_table = {headers[0]:{0,0}}

for x_col in range(X.shape[1]):
    col = X[:,x_col]
    h = headers[x_col]
    
    contingency_table[headers[x_col]] = {}
    print(col)
    for val_x in range(get_max(col)+1):
        count=0
        for val_y in range(get_max(y)+1):
            
            contingency_table[headers[x_col]][val_x,val_y]=  0

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[1 1 0 ... 0 0 0]
[1 1 1 ... 1 0 1]
[1 1 0 ... 0 1 0]
[0 0 0 ... 0 0 0]
[0 1 1 ... 0 1 0]
[0 0 1 ... 1 1 1]
[0 0 1 ... 1 1 0]
[0 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 1 0 ... 0 0 0]
[0 0 0 ... 0 0 1]
[0 0 1 ... 1 1 0]
[0 0 0 ... 0 0 1]
[1 1 1 ... 0 1 0]
[0 0 0 ... 1 0 0]
[0 0 0 ... 0 0 1]
[0 0 0 ... 0 0 0]
[1 1 1 ... 1 1 0]


In [96]:
for x_col in range(X.shape[1]):
    col = X[:,x_col]
    print(headers[x_col])
    
    for i in range(col.shape[0]):
        contingency_table[headers[x_col]][col[i],y[i]] +=1

Fever
Tiredness
Dry-Cough
Difficulty-in-Breathing
Sore-Throat
None_Sympton
Pains
Nasal-Congestion
Runny-Nose
Diarrhea
None_Experiencing
Age_0-9
Age_10-19
Age_20-24
Age_25-59
Age_60+
Gender_Female
Gender_Male
Gender_Transgender
Contact_Dont-Know
Contact_No
Contact_Yes


#### Degrees of Freedom

In [97]:
df ={}

for x_col in range(X.shape[1]):
    col = X[:,x_col]
    print(headers[x_col])
    
    rows = (get_max(col)+1)
    cols = (get_max(y)+1)
    
    df[headers[x_col]] = (rows-1)*(cols-1)
    

Fever
Tiredness
Dry-Cough
Difficulty-in-Breathing
Sore-Throat
None_Sympton
Pains
Nasal-Congestion
Runny-Nose
Diarrhea
None_Experiencing
Age_0-9
Age_10-19
Age_20-24
Age_25-59
Age_60+
Gender_Female
Gender_Male
Gender_Transgender
Contact_Dont-Know
Contact_No
Contact_Yes


Degrees of Freedom for Contingency Tables = (r-1)*(c-1)

In [98]:
print(df)

{'Fever': 3, 'Tiredness': 3, 'Dry-Cough': 3, 'Difficulty-in-Breathing': 3, 'Sore-Throat': 3, 'None_Sympton': 3, 'Pains': 3, 'Nasal-Congestion': 3, 'Runny-Nose': 3, 'Diarrhea': 3, 'None_Experiencing': 3, 'Age_0-9': 3, 'Age_10-19': 3, 'Age_20-24': 3, 'Age_25-59': 3, 'Age_60+': 3, 'Gender_Female': 3, 'Gender_Male': 3, 'Gender_Transgender': 3, 'Contact_Dont-Know': 3, 'Contact_No': 3, 'Contact_Yes': 3}


### Expected Values

In [99]:
expected_values = {headers[0]:{0,0}}

for x_col in range(X.shape[1]):
    col = X[:,x_col]
    h = headers[x_col]
    
    expected_values[headers[x_col]] = {}
    Total = np.sum(list(contingency_table[headers[x_col]].values()))
    rowstotal = []
    colstotal = []
    
    for val_x in range(get_max(col)+1):
        
        for val_y in range(get_max(y)+1):
            rowstotal.append(contingency_table[headers[x_col]][val_x,val_y]) 
            
    for val_y in range(get_max(y)+1):
        
        for val_x in range(get_max(col)+1):
            colstotal.append(contingency_table[headers[x_col]][val_x,val_y])
    
    for val_x in range(get_max(col)+1):
        
        for val_y in range(get_max(y)+1):
            p = (rowstotal[val_x]/Total)*(colstotal[val_y]/Total)
            expected_values[headers[x_col]][val_x,val_y]=  p*Total
            

In [100]:
print(expected_values)

{'Fever': {(0, 0): 935.6268000000001, (0, 1): 415.5398, (0, 2): 895.1854, (0, 3): 419.7782, (1, 0): 895.1854, (1, 1): 397.57856666666663, (1, 2): 856.4920333333332, (1, 3): 401.63376666666665}, 'Tiredness': {(0, 0): 507.00000000000006, (0, 1): 487.63, (0, 2): 483.47, (0, 3): 484.51, (1, 0): 483.47, (1, 1): 464.99896666666666, (1, 2): 461.03203333333335, (1, 3): 462.0237666666667}, 'Dry-Cough': {(0, 0): 377.44083333333333, (0, 1): 480.74633333333327, (0, 2): 364.093, (0, 3): 471.09999999999997, (1, 0): 364.093, (1, 1): 463.7452, (1, 2): 351.21720000000005, (1, 3): 454.44000000000005}, 'Difficulty-in-Breathing': {(0, 0): 468.50003333333336, (0, 1): 487.6199333333334, (0, 2): 463.3764, (0, 3): 467.1254, (1, 0): 463.3764, (1, 1): 482.2872, (1, 2): 458.3088, (1, 3): 462.01680000000005}, 'Sore-Throat': {(0, 0): 906.1932000000002, (0, 1): 423.55060000000003, (0, 2): 886.2062, (0, 3): 407.90860000000004, (1, 0): 886.2062, (1, 1): 414.2087666666667, (1, 2): 866.6600333333332, (1, 3): 398.911766

### Calculate Chi-Square Value

In [101]:
ChiSquare=[]

for x_col in range(X.shape[1]):
    col = X[:,x_col]
    O_E2_divE=[]
    for val_x in range(get_max(col)+1):
        for val_y in range(get_max(y)+1):
            Observed = (contingency_table[headers[x_col]][val_x,val_y])
            Expected = expected_values[headers[x_col]][val_x,val_y]
            O_E = Observed-Expected
            O_E2 = O_E * O_E
            O_E2_divE.append(O_E2/Expected) 
    ChiSquare.append(np.sum(O_E2_divE))

In [102]:
print(ChiSquare)

[168384.33475722797, 178679.82035330965, 224548.69581150825, 183796.39087166867, 169696.80302300825, 847476.2642236493, 158022.19408370458, 206245.6628140825, 205418.30394904967, 159763.70925434006, 576819.4347981558, 255405.17863815947, 254371.45952805958, 252550.4685363993, 256182.32597447364, 251557.21367774432, 164812.12625076916, 164507.8864260789, 163392.77755801764, 165183.30131751084, 163296.80585817815, 164275.56079003363]


### Accept or Reject the Null Hypothesis

With 95% confidence, we will check if the calculated Chi-Square Values will fall in the acceptance or rejection region. If the variables fall in acceptance region (are less than the critical Chi-Square value), we will conclude that the two variables are independent and cannot be selected for training.

In [103]:
drop = []
for h in range(len(headers)):
    print("Testing if variable %s is independent of variable Condition."%headers[h])
    deg = df[headers[h]]
    print("- Degrees of Freedom is %d"%(deg))
    if deg == 3:
        crit_val = 7.82
    if deg == 27:
        crit_val = 40.11
    print("- Critical Value is %d"%(crit_val))
    
    if(ChiSquare[h]>crit_val):
        print("-- Variable %s is dependent on variable Condition."%headers[h])
    else:
        print("-- Variable %s is independent on variable Condition."%headers[h])
        print("DROPPING VARIABLE: %s"%headers[h])
        drop.append(h)
        
X = np.delete(X,drop,1)
    

Testing if variable Fever is independent of variable Condition.
- Degrees of Freedom is 3
- Critical Value is 7
-- Variable Fever is dependent on variable Condition.
Testing if variable Tiredness is independent of variable Condition.
- Degrees of Freedom is 3
- Critical Value is 7
-- Variable Tiredness is dependent on variable Condition.
Testing if variable Dry-Cough is independent of variable Condition.
- Degrees of Freedom is 3
- Critical Value is 7
-- Variable Dry-Cough is dependent on variable Condition.
Testing if variable Difficulty-in-Breathing is independent of variable Condition.
- Degrees of Freedom is 3
- Critical Value is 7
-- Variable Difficulty-in-Breathing is dependent on variable Condition.
Testing if variable Sore-Throat is independent of variable Condition.
- Degrees of Freedom is 3
- Critical Value is 7
-- Variable Sore-Throat is dependent on variable Condition.
Testing if variable None_Sympton is independent of variable Condition.
- Degrees of Freedom is 3
- Critica

---

## Shuffle and Split

In [104]:
def shuffle_and_split(x,y):
    global x_train
    global x_test
    global y_train
    global y_test
    
    N = x.shape[0]
    P = np.arange(0,N,1)
    np.random.shuffle(P)
    lim= int(0.8*N)
    
    x_train = np.copy(x[P[0:lim],:])
    y_train = np.copy(y[P[0:lim]])
    x_test = np.copy(x[P[lim:N],:])
    y_test = np.copy(y[P[lim:N]])

In [105]:
shuffle_and_split(X,y)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(24000, 22)
(24000,)
(6000, 22)
(6000,)


In [106]:
print((x_train[0][-1]))
x_train[0]

0


array([0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
      dtype=object)

In [107]:
pkl_filename_x_train = "pickle_model_x_train.pkl"
with open(pkl_filename_x_train, 'wb') as file:
    pickle.dump(x_train, file)
    
pkl_filename_x_test = "pickle_model_x_test.pkl"
with open(pkl_filename_x_test, 'wb') as file:
    pickle.dump(x_test, file)
    
pkl_filename_y_train = "pickle_model_y_train.pkl"
with open(pkl_filename_y_train, 'wb') as file:
    pickle.dump(y_train, file)
    
pkl_filename_y_test = "pickle_model_y_test.pkl"
with open(pkl_filename_y_test, 'wb') as file:
    pickle.dump(y_test, file)

In [33]:
with open("pickle_model_x_train.pkl", 'rb') as f:
    x_train = pickle.load(f)
    
with open("pickle_model_x_test.pkl", 'rb') as f:
    x_test = pickle.load(f)
    
with open("pickle_model_y_train.pkl", 'rb') as f:
    y_train = pickle.load(f)
    
with open("pickle_model_y_test.pkl", 'rb') as f:
    y_test = pickle.load(f)

---

# Decision Tree

In [ ]:
# Fever_values = []
# Tiredness_values = []
# Dry_Cough_values=[]
# Difficulty_Breathing_values = []
# Sore_Throat_values = []
# No_Symptoms_values=[]
# Pains_values = []
# Nasal_Congestion_values = []
# Runny_Nose_values=[]
# Diarrhea_values = []
# None_Experiencing_Values = []
# Age_0_9_Values =[]
# Age_10_19_Values =[]
# Age_20_24_Values =[]
# Age_25_59_Values = []
# Age_60_pls_Values = []
# Gender_Female_Values =[]
# Gender_Male_Values =[]
# Gender_Trans_Values =[]
# Contact_dk_Values = []
# Contact_no_Values =[]
# Contact_yes_Values =[]
# Country_Values = []

In [ ]:
#
# classes = [0,1,2,3]
# features = headers
# #

# classes2code = {}
# code2classes = {}
# i=0
# for v in classes:
#     print(v)
#     classes2code[v]=i
#     code2classes[i]=v
#     i+=1
# #
# print(classes2code)

In [ ]:
# #
# # convert dataset to numpy arrays
# n = len(headers)  # number of features
# m = len(x_train)  # number of instances
# X = np.zeros([m,n],dtype=float)
# Y = np.zeros([m],dtype=float)
# for i in range(m):
#     X[i,0] = x_train[i][0]
#     X[i,1] = x_train[i][1]
#     X[i,2] = x_train[i][2]
#     X[i,3] = x_train[i][3]
#     X[i,4] = x_train[i][4]
#     X[i,5] = x_train[i][5]
#     X[i,6] = x_train[i][6]
#     X[i,7] = x_train[i][7]
#     X[i,8] = x_train[i][8]
#     X[i,9] = x_train[i][9]
#     X[i,10] = x_train[i][10]
#     X[i,11] = x_train[i][11]
#     X[i,12] = x_train[i][12]
#     X[i,13] = x_train[i][13]
#     X[i,14] = x_train[i][14]
#     X[i,15] = x_train[i][15]
#     X[i,16] = x_train[i][16]
#     X[i,17] = x_train[i][17]
#     X[i,18] = x_train[i][18]
#     X[i,19] = x_train[i][19]
#     X[i,20] = x_train[i][20]
#     X[i,21] = x_train[i][21]
#     X[i,22] = x_train[i][22]
#     Y[i] = y_train[i]

In [ ]:
# print (X)
# print (Y)

In [ ]:
# X.shape

In [ ]:
# Y.reshape(1,24000).ravel()

In [ ]:
# dt_model = DecisionTreeClassifier(criterion='entropy', )
# dt_model.fit(X,Y)

In [ ]:
# param_DT = {'criterion':['entropy','gini'], 
#             'splitter': ['best','random'],
#             'max_depth': [10, 15,20,25,30],
#            'max_features': ['auto','sqrt', 'log2'],
#            'min_samples_split': [5,7,10,15,20],
#             'min_samples_leaf':[5,7,10,15,20],
#             'max_leaf_nodes': [5,7,10,15,20],
#             'min_impurity_decrease': [5,7,10,15,20],
#             'class_weight': ['balanced']
            
#            }
# DT = DecisionTreeClassifier()
# clf_2 = GridSearchCV(DT, param_DT, n_jobs=-1 ,cv=5, verbose = 100)

In [ ]:
# %time clf_2.fit(X,Y) 

In [ ]:
# print(clf_2.best_params_) 

In [ ]:
# y_pred_dt = clf_2.predict(x_test)
# acc_dt =accuracy_score(y_test, y_pred_dt)
# # acc_dt 

In [ ]:
# # predict and compare (you should check consistency)
# print('y=',dt_model.predict(x_test))
# print('Y=',Y)

In [ ]:
# dot_data = tree.export_graphviz(dt_model, out_file=None,
#                                 feature_names=headers, 
#                                 class_names = str(classes))
# graph = graphviz.Source(dot_data) 
# %time graph.render("Assignment", view=True) 

____

##  Decision Tree from Scratch

In [ ]:
class DecisionTreeClassifier(object):
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
    
    def fit(self, x, y, par_node={}, depth=0):
        if par_node is None: 
            return None
        elif len(y) == 0:
            return None
        elif self.all_same(y):
            return {'val':y[0]}
        elif depth >= self.max_depth:
            return None
        else: 
            col, cutoff, entropy = self.find_best_split_of_all(x, y)    # find one split given an information gain 
            y_left = y[x[:, col] < cutoff]
            y_right = y[x[:, col] >= cutoff]
            par_node = {'col': headers[col], 'index_col':col,
                        'cutoff':cutoff,
                       'val': np.round(np.mean(y))}
            par_node['left'] = self.fit(x[x[:, col] < cutoff], y_left, {}, depth+1)
            par_node['right'] = self.fit(x[x[:, col] >= cutoff], y_right, {}, depth+1)
            self.depth += 1 
            self.trees = par_node
            return par_node
    
    def find_best_split_of_all(self, x, y):
        col = None
        min_entropy = 1
        cutoff = None
        for i, c in enumerate(x.T):
            entropy, cur_cutoff = self.find_best_split(c, y)
            if entropy == 0:    # find the first perfect cutoff. Stop Iterating
                return i, cur_cutoff, entropy
            elif entropy <= min_entropy:
                min_entropy = entropy
                col = i
                cutoff = cur_cutoff
        return col, cutoff, min_entropy
    
    def find_best_split(self, col, y):
        min_entropy = 10
        n = len(y)
        for value in set(col):
            y_predict = col < value
            my_entropy = get_entropy(y_predict, y)
            if my_entropy <= min_entropy:
                min_entropy = my_entropy
                cutoff = value
        return min_entropy, cutoff
    
    def all_same(self, items):
        return all(x == items[0] for x in items)
                                           
    def predict(self, x):
        tree = self.trees
        results = np.array([0]*len(x))
        for i, c in enumerate(x):
            results[i] = self._get_prediction(c)
        return results
    
    def _get_prediction(self, row):
        cur_layer = self.trees
        while cur_layer.get('cutoff'):
            if row[cur_layer['index_col']] < cur_layer['cutoff']:
                cur_layer = cur_layer['left']
            else:
                cur_layer = cur_layer['right']
        else:
            return cur_layer.get('val')

In [ ]:
def entropy_func(c, n):#C -> Class, n -> number of samples, e-> entropy (math formula)
    e = -(c*1.0/n)*math.log(c*1.0/n, 2)
    return e

def entropy_cal(c1, c2): #Calculates entropy when there are multiple classes """Returns entropy of a group of data c1: count of one classc2: count of another class
   
    if c1== 0 or c2 == 0:  # when there is only one class in the group, entropy is 0
        return 0
    e = entropy_func(c1, c1+c2) + entropy_func(c2, c1+c2)
    return e

# # get the entropy of one big circle showing above
def entropy_of_one_division(division): 
    """
    Returns entropy of a divided group of data
    Data may have multiple classes
    """
    s = 0
    n = len(division)
    classes = set(division)
    for c in classes:   # for each class, get entropy
        n_c = sum(division==c)
        e = n_c*1.0/n * entropy_cal(sum(division==c), sum(division!=c)) # weighted avg
        s += e
    return s, n

# The whole entropy of two big circles combined
def get_entropy(y_predict, y_real):
    """
    Returns entropy of a split
    y_predict is the split decision, True/Fasle, and y_true can be multi class
    """
    if len(y_predict) != len(y_real):
        print('They have to be the same length')
        return None
    n = len(y_real)
    s_true, n_true = entropy_of_one_division(y_real[y_predict]) # left hand side entropy
    s_false, n_false = entropy_of_one_division(y_real[~y_predict]) # right hand side entropy
    s = n_true*1.0/n * s_true + n_false*1.0/n * s_false # overall entropy, again weighted average
    return s

In [ ]:
# DT_Params = {'criterion':['entropy','gini'], 
#             'splitter': ['best','random'],
#             'max_depth': [10, 15,20,25,30],
#            'max_features': ['auto','sqrt', 'log2'],
#            'min_samples_split': [5,7,10,15,20],
#             'min_samples_leaf':[5,7,10,15,20],
#             'max_leaf_nodes': [5,7,10,15,20],
#             'min_impurity_decrease': [5,7,10,15,20],
#             'class_weight': ['balanced']}
            
# DT_Params['criterion']

___
##### Decision Tree Parameter Grid

In [ ]:
criterion = ['entropy','gini'] 
splitter = ['best','random']
max_depth = np.linspace(3, 25, num = 6).astype(int)
max_features = ['auto','sqrt', 'log2']
min_samples_split = np.linspace(3, 15, num = 6).astype(int)
min_samples_leaf =np.linspace(3, 15, num = 6).astype(int)
max_leaf_nodes = np.linspace(3, 15, num = 6).astype(int)
min_impurity_decrease = np.linspace(3, 15, num = 6).astype(int)
class_weight =['balanced']

___

##### CrossValidation Function: 5Fold

In [ ]:
def cvsplit(x,y):
    
    N = x.shape[0]
    print(N)
    P = np.arange(0,N,1)
    np.random.shuffle(P)
    lim_1= int(0.2*N)
    lim_2= int(0.4*N)
    lim_3= int(0.6*N)
    lim_4= int(0.8*N)
    
    
    x_1= np.copy(x[P[0:lim_1],:])
    x_2= np.copy(x[P[lim_1:lim_2],:])
    x_3= np.copy(x[P[lim_2:lim_3],:])
    x_4= np.copy(x[P[lim_3:lim_4],:])
    x_5= np.copy(x[P[lim_4:N],:])
    
    y_1= np.copy(y[P[0:lim_1]])
    y_2= np.copy(y[P[lim_1:lim_2]])
    y_3= np.copy(y[P[lim_2:lim_3]])
    y_4= np.copy(y[P[lim_3:lim_4]])
    y_5= np.copy(y[P[lim_4:N]])
    
    return x_1,x_2,x_3,x_4,x_5,y_1,y_2,y_3,y_4,y_5

___
##### Accuracy

In [ ]:
# # Accuracy

# true_positives = 0

# for i in range(0, len(lr_probs)):
    
#     if lr_probs[i,0] > 0.5 and testy[i] == 0:
#         true_positives += 1
#     elif lr_probs[i,1] > 0.5 and testy[i] == 1:
#         true_positives += 1

# accuracy = true_positives/len(testy)

# print(accuracy)

___

##### GridSearch

In [ ]:
# %%time

# Accuracy_Scores = {}
# for c in criterion:
#     for s in splitter:
#         for d in max_depth:
#             for f in max_features:
#                 for ms in min_samples_split:
#                     for l min_samples_leaf:
#                         for i in min_impurity_decrease:
#                                 dbscan = DecisionTreeClassifier().fit(x)
            
#             dbscan
#             Accuracy_Scores[str(e) + ',' +str(m) + ',' +str(a)] = accuracy_score(y_true = y,y_pred=(dbscan.labels_+1))

In [ ]:
%%time

Accuracy_Scores = {}
for d in max_depth:
    print("Testing for Max Depth = " +str(d))
    
    clf = DecisionTreeClassifier(max_depth=d)
    
    print(x_train.shape,y_train.shape)
    
    print(" - Splitting into 5 folds")
    x_1,x_2,x_3,x_4,x_5,y_1,y_2,y_3,y_4,y_5 = cvsplit(x_train,y_train)
    print(x_1.shape)
    train1 =[[]]
    test1 =[]
    train2 =[[]]
    test2 =[]
    train3 =[[]]
    test3 =[]
    train4 =[[]]
    test4 =[]
    train5 =[[]]
    test5 =[]
    
    train1.extend((list(x_2)+list(x_3))+(list(x_4)+list(x_5)))
    test1.extend((y_2+y_3)+(y_4+y_5))
    train2.extend((x_1+x_3)+(x_4+x_5))
    test2.extend((y_1+y_3)+(y_4+y_5))
    train3.extend((x_2+x_1)+(x_4+x_5))
    test3.extend((y_2+y_1)+(y_4+y_5))
    train4.extend((x_2+x_3)+(x_1+x_5))
    test4.extend((y_2+y_3)+(y_1+y_5))
    train5.extend((x_2+x_3)+(x_4+x_1))
    test5.extend((y_2+y_3)+(y_4+y_1))
    
    train1 = np.array(train1)
    test1 = np.array(test1)
    train2 = np.array(train2)
    test2 = np.array(test2)
    train3 = np.array(train3)
    test3 = np.array(test3)
    train4 = np.array(train4)
    test4 = np.array(test4)
    train5 = np.array(train5)
    test5 = np.array(test5)
    
    clf.fit(train1,test1)
    dtc_1_score = accuracy_score(y_1, clf.predict(x_1))
    print("  -  Trained 1/5 Folds. Last Fold Accuracy was: %s" %(str(dtc_1_score)))
    
    clf.fit(train2,test2)
    dtc_2_score = accuracy_score(y_2, clf.predict(x_2))
    print("  -  Trained 2/5 Folds. Last Fold Accuracy was: %s" %(str(dtc_2_score)))

    clf.fit(train3,test3)
    dtc_3_score = accuracy_score(y_3, clf.predict(x_3))
    print("  -  Trained 3/5 Folds. Last Fold Accuracy was: " %( str(dtc_3_score)))
    
    clf.fit(train4,test4)
    dtc_4_score = accuracy_score(y_4, clf.predict(x_4))
    print("  -  Trained 4/5 Folds. Last Fold Accuracy was: " %( str(dtc_4_score)))
    
    clf.fit(train5,test5)
    dtc_5_score = accuracy_score(y_5, clf.predict(x_5))
    print("  -  Trained 5/5 Folds. Last Fold Accuracy was: " %( str(dtc_5_score)))
    
    Average_Score = (dtc_1_score+dtc_2_score+dtc_3_score+dtc_4_score+dtc_5_score)/5
    print("   -   Average Accuracy achieved for max depth %s was: %s" %(d, str(Average_Score)))
    
    Accuracy_Scores[str(d)] = Average_Score
                
#     Accuracy_Scores[str(d)] = (dtc_1_4_score+dtc_1_5_score+dtc_2_4_score+dtc_2_5_score+dtc_3_4_score+dtc_3_5_score)/6

In [ ]:
print(Accuracy_Scores)
keys = list(Accuracy_Scores.keys())
vals = list(Accuracy_Scores.values())
 
# print key with val 100
position = vals.index(np.max(list(Accuracy_Scores.values())))
print()
print('The best score was achieved with a max depth of:',keys[position])

___

In [ ]:
clf = DecisionTreeClassifier(max_depth = 20 ).fit(x_train,y_train)
pred = clf.predict(x_test)

accuracy_score(y_test, pred)

___

In [ ]:
clf = DecisionTreeClassifier()
%time clf.fit(x_train, y_train)

In [ ]:
y_pred_dt = clf.predict(x_test)
acc_dt =accuracy_score(y_test, y_pred_dt)
acc_dt 

## Decision Tree

In [115]:
def Pure_Output(data):
    if len(np.unique(y_train)) == 1:
        return True
    else:
        return False
print(headers)   

['Fever', 'Tiredness', 'Dry-Cough', 'Difficulty-in-Breathing', 'Sore-Throat', 'None_Sympton', 'Pains', 'Nasal-Congestion', 'Runny-Nose', 'Diarrhea', 'None_Experiencing', 'Age_0-9', 'Age_10-19', 'Age_20-24', 'Age_25-59', 'Age_60+', 'Gender_Female', 'Gender_Male', 'Gender_Transgender', 'Contact_Dont-Know', 'Contact_No', 'Contact_Yes', 'Country']


    for i in range (len(x_train)):
        check purity of each feature

        choose highest information gain
            print(i)
        Set Rootnode as header of i
            split on i
repeat

In [116]:
rows, cols = x_train.shape 

In [117]:
# Counting number of output classes
sev_0 = 0
sev_1 = 0
sev_2 = 0
sev_3 = 0

for i in y_train:
    if i == 0:
        sev_0 +=1
    elif i == 1:
        sev_1 +=1
    elif i == 2:
        sev_2 +=1
    else:
        sev_3 +=1
print('Sev_0 = ',sev_0)
print('Sev_1 = ',sev_1)
print('Sev_2 = ',sev_2)
print('Sev_3 = ',sev_3)
print(sev_0 + sev_1 + sev_2 + sev_3, len(y_train))

Sev_0 =  6106
Sev_1 =  5942
Sev_2 =  5991
Sev_3 =  5961
24000 24000


In [118]:
# def entropy(target_num ,rows):
e_0 = -((sev_0)*(1/rows)*np.log2((sev_0)*(1/rows)))
e_1 = -((sev_1)*(1/rows)*np.log2((sev_1)*(1/rows)))
e_2 = -((sev_2)*(1/rows)*np.log2((sev_2)*(1/rows)))
e_3 = -((sev_3)*(1/rows)*np.log2((sev_3)*(1/rows)))
ent = e_0 + e_1 + e_2 + e_3 
ent

1.9999190956165762

In [119]:
def entropy(target_num ,instances):
    e = -((target_num)*(1/instances)*np.log2((target_num)*(1/instances)))
    return e

data_entropy= entropy(sev_0, rows)+entropy(sev_1, rows)+entropy(sev_2, rows)+entropy(sev_3, rows)
data_entropy

1.9999190956165762

In [120]:
# computing the remainder
t_fever = 0
f_fever = 0
print(headers[0])
for i in x_train[:,0]:
    
    if i == 1:
        t_fever += 1
    else:
        f_fever += 1

print(t_fever, f_fever)
print(len(x_train[:,0]), (t_fever+f_fever))
#___

Fever
7576 16424
24000 24000


In [121]:
# Remainder tuples
true = 0
false = 0
tmp = ()
rem_t_f = []
c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9 = 0,0,0,0,0,0,0,0,0,0


for i in range(cols):
    print(headers[i])
    if i != 22:
        for j in x_train[:,i]:
            if j == 1:
                true += 1
            else:
                false += 1
        print('True  False')
        print(true, false)
        print()
        tmp = (true, false)
        rem_t_f.append(tmp)
        tmp = ()
        true, false = 0, 0
    else: 
        for j in x_train[:,i]:
            if j == 0:
                c_0 += 1
            elif j == 1:
                c_1 += 1
            elif j == 2:
                c_2 += 1
            elif j == 3:
                c_3 += 1
            elif j == 4:
                c_4 += 1
            elif j == 5:
                c_5 += 1
            elif j == 6:
                c_6 += 1
            elif j == 7:
                c_7 += 1
            elif j == 8:
                c_8 += 1
            else:
                c_9 += 1
        print('C0   C1   C2   C3   C4   C5   C6   C7   C8   C9')
        print(c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9  )
        print()
        tmp = (c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9)
        rem_t_f.append((c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9))
        tmp = ()
        c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9  =0,0,0,0,0,0,0,0,0,0
        
print(len(rem_t_f))

# print(t_tired, f_tired)
# print(len(x_train[:,1]), (t_tired+f_tired))

Fever
True  False
7576 16424

Tiredness
True  False
11966 12034

Dry-Cough
True  False
13557 10443

Difficulty-in-Breathing
True  False
12070 11930

Sore-Throat
True  False
7591 16409

None_Sympton
True  False
1483 22517

Pains
True  False
8705 15295

Nasal-Congestion
True  False
12995 11005

Runny-Nose
True  False
13096 10904

Diarrhea
True  False
8781 15219

None_Experiencing
True  False
2165 21835

Age_0-9
True  False
4817 19183

Age_10-19
True  False
4755 19245

Age_20-24
True  False
4789 19211

Age_25-59
True  False
4686 19314

Age_60+
True  False
4953 19047

Gender_Female
True  False
7930 16070

Gender_Male
True  False
8036 15964

Gender_Transgender
True  False
8034 15966

Contact_Dont-Know
True  False
7974 16026

Contact_No
True  False
8013 15987

Contact_Yes
True  False
8013 15987

Country
C0   C1   C2   C3   C4   C5   C6   C7   C8   C9
2344 2470 2375 2398 2462 2389 2371 2446 2379 2366

23


In [122]:
tfever_0 = 0
tfever_1 = 0
tfever_2 = 0
tfever_3 = 0
ffever_0 = 0
ffever_1 = 0
ffever_2 = 0
ffever_3 = 0

for i in range(rows):
    if x_train[i,0] == 1:
        if y_train[i] == 0:
            tfever_0 += 1
        elif y_train[i] == 1:
            tfever_1 += 1
        elif y_train[i] == 2:
            tfever_2 += 1
        else:
            tfever_3 += 1
    elif x_train[i,0] == 0:
        if y_train[i] == 0:
            ffever_0 += 1
        elif y_train[i] == 1:
            ffever_1 += 1
        elif y_train[i] == 2:
            ffever_2 += 1
        else:
            ffever_3 += 1

t,f = tfever_0+tfever_1+tfever_2+tfever_3, ffever_0+ffever_1+ffever_2+ffever_3
# 903, 1884, 1892, 1895, 4223, 4058, 4125, 4020)], [(7574, 16426)

In [123]:
lst_1[0][0]

NameError: name 'lst_1' is not defined

In [124]:
#Remainder per class


rem_fever_t = rem_t_f[0][0]/rows*(-((tfever_0)*(1/t)*np.log2((tfever_0)*(1/t)))-((tfever_1)*(1/t)*np.log2((tfever_1)*(1/t)))-((tfever_2)*(1/t)*np.log2((tfever_2)*(1/t)))-((tfever_3)*(1/t)*np.log2((tfever_3)*(1/t))))
rem_fever_f = rem_t_f[0][1]/rows*(-((ffever_0)*(1/f)*np.log2((ffever_0)*(1/f)))-((ffever_1)*(1/f)*np.log2((ffever_1)*(1/f)))-((ffever_2)*(1/f)*np.log2((ffever_2)*(1/f)))-((ffever_3)*(1/f)*np.log2((ffever_3)*(1/f))))
rem_f = rem_fever_t+rem_fever_f
rem_f

1.9998972524932135

In [125]:
e_t0 = entropy(lst_1[0][0],lst_2[0][0])
e_t1 = entropy(lst_1[0][1],lst_2[0][0])
e_t2 = entropy(lst_1[0][2],lst_2[0][0])
e_t3 = entropy(lst_1[0][3],lst_2[0][0])
e_f0 = entropy(lst_1[0][4],lst_2[0][1])
e_f1 = entropy(lst_1[0][5],lst_2[0][1])
e_f2 = entropy(lst_1[0][6],lst_2[0][1])
e_f3 = entropy(lst_1[0][7],lst_2[0][1])

NameError: name 'lst_1' is not defined

In [126]:
rem__t = rem_t_f[0][0]/rows*(e_t0+e_t1+e_t2+ e_t3)
rem__f = rem_t_f[0][1]/rows*(e_f0+e_f1+e_f2+ e_f3)
rem__f+ rem__t


NameError: name 'e_t0' is not defined

In [127]:
# Remainder Calculator
def remainder(x):
    rem = []
    for i in range(x):
        e_t0 = entropy(lst_1[i][0],lst_2[i][0])
        e_t1 = entropy(lst_1[i][1],lst_2[i][0])
        e_t2 = entropy(lst_1[i][2],lst_2[i][0])
        e_t3 = entropy(lst_1[i][3],lst_2[i][0])
        e_f0 = entropy(lst_1[i][4],lst_2[i][1])
        e_f1 = entropy(lst_1[i][5],lst_2[i][1])
        e_f2 = entropy(lst_1[i][6],lst_2[i][1])
        e_f3 = entropy(lst_1[i][7],lst_2[i][1])
        rem__t = rem_t_f[i][0]/rows*(e_t0+e_t1+e_t2+ e_t3)
        rem__f = rem_t_f[i][1]/rows*(e_f0+e_f1+e_f2+ e_f3)
        rem.append(rem__f+ rem__t)
    return rem

        
rem = remainder_1(cols, rows,x_train,y_train)

NameError: name 'remainder_1' is not defined

In [128]:
Info_Gain = []
for i in range(len(rem)):
#     print('Feature' ,headers[i],'remainder is:',rem[i])
    IG = (data_entropy - rem[i])
    Info_Gain.append(IG)
#     print('Feature' ,headers[i],'information gain of:',IG)
#     print()
for i in range(len(Info_Gain)):
    if Info_Gain[i] == max(Info_Gain):
        print('The best feature to split on is feature',i, headers[i], 'with an information gain of', Info_Gain[i])

NameError: name 'rem' is not defined

In [129]:
print(np.unique(x_train[:,22]))
set_0 = []
set_1 = []
set_2 = []
set_3 = []
set_4 = []
set_5 = []
set_6 = []
set_7 = []
set_8 = []
set_9 = []
set_y_0 = []
set_y_1 = []
set_y_2 = []
set_y_3 = []
set_y_4 = []
set_y_5 = []
set_y_6 = []
set_y_7 = []
set_y_8 = []
set_y_9 = []

for i in range(rows):
    if x_train[i,22] == 0:
        set_0.append(x_train[i,:])
        set_y_0.append(y_train[i])
    elif x_train[i,22] == 1:
        set_1.append(x_train[i,:])
        set_y_1.append(y_train[i])
    elif x_train[i,22] == 2:
        set_2.append(x_train[i,:])
        set_y_2.append(y_train[i])
    elif x_train[i,22] == 3:
        set_3.append(x_train[i,:])
        set_y_3.append(y_train[i])
    elif x_train[i,22] == 4:
        set_4.append(x_train[i,:])
        set_y_4.append(y_train[i])
    elif x_train[i,22] == 5:
        set_5.append(x_train[i,:])
        set_y_5.append(y_train[i])
    elif x_train[i,22] == 6:
        set_6.append(x_train[i,:])
        set_y_6.append(y_train[i])
    elif x_train[i,22] == 7:
        set_7.append(x_train[i,:])
        set_y_7.append(y_train[i])
    elif x_train[i,22] == 8:
        set_8.append(x_train[i,:]) 
        set_y_8.append(y_train[i])
    else:
        set_9.append(x_train[i,:])
        set_y_9.append(y_train[i])
        


[0 1 2 3 4 5 6 7 8 9]


In [130]:
Pure_Output(set_y_0),Pure_Output(set_y_1),Pure_Output(set_y_2),Pure_Output(set_y_3),Pure_Output(set_y_4),Pure_Output(set_y_5),Pure_Output(set_y_6),Pure_Output(set_y_7),Pure_Output(set_y_8),Pure_Output(set_y_9)

(False, False, False, False, False, False, False, False, False, False)

# ACTUAL CODE

In [131]:
def Pure_Output(data):
    if len(np.unique(data)) == 1:
        return True
    else:
        return False

In [132]:
def Entropy(target_num ,instances): #Computes the entropy of a set
    e = -((target_num)*(1/instances)*np.log2((target_num)*(1/instances)))
    return e

In [133]:
def Remainder(feat_num, inst_num, in_set, out_set): #Computes the remainder after splitting over descriptive features
    true = 0
    false = 0
    tmp = ()
    rem_t_f = []
    
    for i in range(feat_num):
#         print(headers[i])
        for j in in_set[:,i]:
            if j == 1:
                true += 1
            else:
                false += 1
#         print('True  False')
#         print(true, false)
#         print()
        tmp = (true, false)
        rem_t_f.append(tmp)
        tmp = ()
        true, false = 0, 0
           
        """ """
    t0 = 0
    t1 = 0
    t2 = 0
    t3 = 0
    f0 = 0
    f1 = 0
    f2 = 0
    f3 = 0
    tmp = ()
    lst_1 = []
    lst_2 = []

    for j in range(feat_num):
#         print(headers[j])
        for i in range(inst_num):
            if in_set[i,j] == 1: #when feature is true
                if out_set[i] == 0: #check corresponding output class
                    t0 += 1
                elif out_set[i] == 1:
                    t1 += 1
                elif out_set[i] == 2:
                    t2 += 1
                else:
                    t3 += 1
            elif in_set[i,j] == 0:
                if out_set[i] == 0:
                    f0 += 1
                elif out_set[i] == 1:
                    f1 += 1
                elif out_set[i] == 2:
                    f2 += 1
                else:
                    f3 += 1
        tmp = (t0,t1,t2,t3,f0,f1,f2,f3)
        lst_1.append(tmp)
        tmp = ()
        tmp = ((t0+t1+t2+t3),(f0+f1+f2+f3))
        lst_2.append(tmp)
        tmp = ()
        t0,t1,t2,t3,f0,f1,f2,f3 = 0,0,0,0,0,0,0,0


    """          """
    rem = []
    
   
    for i in range(feat_num):
        e_t0 = entropy(lst_1[i][0],lst_2[i][0])
        e_t1 = entropy(lst_1[i][1],lst_2[i][0])
        e_t2 = entropy(lst_1[i][2],lst_2[i][0])
        e_t3 = entropy(lst_1[i][3],lst_2[i][0])
        e_f0 = entropy(lst_1[i][4],lst_2[i][1])
        e_f1 = entropy(lst_1[i][5],lst_2[i][1])
        e_f2 = entropy(lst_1[i][6],lst_2[i][1])
        e_f3 = entropy(lst_1[i][7],lst_2[i][1])
        rem__t = rem_t_f[i][0]/rows*(e_t0+e_t1+e_t2+ e_t3)
        rem__f = rem_t_f[i][1]/rows*(e_f0+e_f1+e_f2+ e_f3)
        rem.append(rem__f+ rem__t)
    return rem


In [134]:
def Information_Gain(rem_list, data_entropy): #takes list of remainders and calculates the best attribute to split on
    Info_Gain = []
    for i in range(len(rem_list)):
    #     print('Feature' ,headers[i],'remainder is:',rem[i])
        IG = (data_entropy - rem_list[i])
        Info_Gain.append(IG)
    #     print('Feature' ,headers[i],'information gain of:',IG)
    #     print()
    for i in range(len(Info_Gain)):
        if Info_Gain[i] == max(Info_Gain):
            feature = (i,Info_Gain[i] , headers[i])
    return feature

In [135]:
print(np.unique(x_train[:,22]))
set_0 = []
set_1 = []
set_2 = []
set_3 = []
set_4 = []
set_5 = []
set_6 = []
set_7 = []
set_8 = []
set_9 = []
set_y_0 = []
set_y_1 = []
set_y_2 = []
set_y_3 = []
set_y_4 = []
set_y_5 = []
set_y_6 = []
set_y_7 = []
set_y_8 = []
set_y_9 = []

for i in range(rows):
    if x_train[i,22] == 0:
        set_0.append(x_train[i,:])
        set_y_0.append(y_train[i])
    elif x_train[i,22] == 1:
        set_1.append(x_train[i,:])
        set_y_1.append(y_train[i])
    elif x_train[i,22] == 2:
        set_2.append(x_train[i,:])
        set_y_2.append(y_train[i])
    elif x_train[i,22] == 3:
        set_3.append(x_train[i,:])
        set_y_3.append(y_train[i])
    elif x_train[i,22] == 4:
        set_4.append(x_train[i,:])
        set_y_4.append(y_train[i])
    elif x_train[i,22] == 5:
        set_5.append(x_train[i,:])
        set_y_5.append(y_train[i])
    elif x_train[i,22] == 6:
        set_6.append(x_train[i,:])
        set_y_6.append(y_train[i])
    elif x_train[i,22] == 7:
        set_7.append(x_train[i,:])
        set_y_7.append(y_train[i])
    elif x_train[i,22] == 8:
        set_8.append(x_train[i,:]) 
        set_y_8.append(y_train[i])
    else:
        set_9.append(x_train[i,:])
        set_y_9.append(y_train[i])
        


[0 1 2 3 4 5 6 7 8 9]


In [137]:
def feature_split(x):
    set_0 = []
set_1 = []
    for i in range(rows):
        if x_train[i,x] == 0:
            set_0.append(x_train[i,:])
            set_y_0.append(y_train[i])
        elif x_train[i,x] == 1:
            set_1.append(x_train[i,:])
            set_y_1.append(y_train[i])
    return set_0, set_y_0, set_1, set_y_1

IndentationError: unexpected indent (<ipython-input-137-3b03b929204d>, line 4)

____

# Neural Network

## Test

## Activation Function Definitions

In [108]:
def sigmoid(x):
  
  #return 1/(1 + np.exp(-x))
  return np.tanh(x)

def sigmoid_prime(x):
  
  #return sigmoid(x)*(1 - sigmoid(x))
  return 1 - np.power(np.tanh(x), 2)

In [109]:
def dot_product_list(a,b):
    dot_p = 0
    for i in range(len(a)):
        dot_p+= a[i]*b[i]
        
    return dot_p

In [110]:
def dot_product_np(a,b):
    dot_p = 0
    for i in range(a.shape[0]):
        dot_p+= a[i]*b[i]
        
    return dot_p

## Weights for NN

In [111]:
print(y_train)

[0 3 3 ... 1 2 1]


In [112]:
Sev_none_train = []
Sev_mild_train = []
Sev_mod_train = []
Sev_sev_train = []

for y in y_train:
    if y==0:
        Sev_none_train.append(1)
        Sev_mild_train.append(0)
        Sev_mod_train.append(0)
        Sev_sev_train.append(0)
    if y==1:
        Sev_none_train.append(0)
        Sev_mild_train.append(1)
        Sev_mod_train.append(0)
        Sev_sev_train.append(0)
    if y==2:
        Sev_none_train.append(0)
        Sev_mild_train.append(0)
        Sev_mod_train.append(1)
        Sev_sev_train.append(0)
    if y==3:
        Sev_none_train.append(0)
        Sev_mild_train.append(0)
        Sev_mod_train.append(0)
        Sev_sev_train.append(1)


    

In [113]:
Sev_none_test = []
Sev_mild_test = []
Sev_mod_test = []
Sev_sev_test = []

for y in y_test:
    if y==0:
        Sev_none_test.append(1)
        Sev_mild_test.append(0)
        Sev_mod_test.append(0)
        Sev_sev_test.append(0)
    if y==1:
        Sev_none_test.append(0)
        Sev_mild_test.append(1)
        Sev_mod_test.append(0)
        Sev_sev_test.append(0)
    if y==2:
        Sev_none_test.append(0)
        Sev_mild_test.append(0)
        Sev_mod_test.append(1)
        Sev_sev_test.append(0)
    if y==3:
        Sev_none_test.append(0)
        Sev_mild_test.append(0)
        Sev_mod_test.append(0)
        Sev_sev_test.append(1)


    

In [146]:
n = x_train.shape[1]
m = 4
num_hidden_layer_nodes = n-1

In [173]:
print(n)

22


In [178]:
#weights between input and hidden nodes
w_0= (2*np.random.random((n,num_hidden_layer_nodes)))
#weights between hidden and output nodes
w_1 = (2*np.random.random((num_hidden_layer_nodes,m)))
#bias between input and hidden nodes
b_0 =2*np.random.random(num_hidden_layer_nodes)-1
#bias between hidden and output nodes
b_1 =2*np.random.random(m)-1
#learning rate
alpha = 0.1


## Training

In [180]:
H = [] 
Neu_out = 0
# for n_input, weight in zip(x_train[0],w_0):
#     Neu_out += n_input*weight
error = []
for iter in range(5000):
    
    print("Starting Iteration %f/5000, Training is %f complete"%(iter,(iter/5000*100)))
    
    x_row = (np.random.randint(low=0,high=x_train.shape[0]))
    
    # FEEDFORWARD
    
    H = []
    H_row = []
    Neu_out = 0
    Z_inj = []

    for j in range(num_hidden_layer_nodes):
        Neu_out=0
        for i in range(n):
            Neu_out += x_train[x_row,i] * w_0[i,j] #SUM of x[0]*w_0[0]+x[1]*w_0[1]...
        Neu_out +=b_0[j]
        
        Z_inj.append(Neu_out)
        
        H.append(sigmoid(Neu_out)) #output of hidden layer nodes
                                    #H[j] -> j = Hidden Layer Neuron Index
    
    Y=[]
    Y_row = []
    Neu_out = 0


# # for n_input, weight in zip(x_train[0],w_0):
# #     Neu_out += n_input*weight
#     for k in range(len(H)):
    #for j in range(w_1.shape[0]):
    for k  in range(m):
        Neu_out = 0
        for j in range(num_hidden_layer_nodes):
            Neu_out+=H[j]*w_1[j,k]
        Neu_out +=b_1[k]
        
        Y.append(sigmoid(Neu_out)) #output of output nodes
                                    #Y[k] -> k = Output Layer Neuron index
    
    #BACKPROP
    delta_k = [0,0,0,0]
    
    delta_k[0] = (Sev_none_train[x_row] - Y[0])*sigmoid_prime(Y[0])
    delta_k[1] = (Sev_mild_train[x_row] - Y[1])*sigmoid_prime(Y[1])
    delta_k[2] = (Sev_mod_train[x_row] - Y[2])*sigmoid_prime(Y[2])
    delta_k[3] = (Sev_sev_train[x_row] - Y[3])*sigmoid_prime(Y[3])
    
    w_1_corr = np.zeros((num_hidden_layer_nodes,m))
    b_1_corr = []
    
    for j in range(num_hidden_layer_nodes):
        for k in range(m):
            w_1_corr[j,k]=alpha*delta_k[k]*H[j] #Correcting factor for w_1 (Weights of Hidden Nodes)
    
    for k in range(m):
        b_1_corr.append(alpha*delta_k[k]) #Correcting factor for b_1 (Biases for Hidden Nodes)
    
    delta_in_j = []
    for j in range(num_hidden_layer_nodes):
        d_inj = 0
        for k in range(m):
            d_inj += delta_k[k]*w_1[j,k]
        delta_in_j.append(d_inj)
    
    for j in range(num_hidden_layer_nodes):
        delta_j.append(delta_in_j[j]*sigmoid_prime(Z_inj[j]))
    
    
    w_0_corr = np.zeros((n,num_hidden_layer_nodes))
    for i in range(n):
        for j in range(num_hidden_layer_nodes):
            w_0_corr[i,j] = alpha*delta_j[j]*x_train[x_row,i] #Correcting factor for w_0 (Weights of Input Nodes)
            
    b_0_corr = []
    for j in range(num_hidden_layer_nodes):
        b_0_corr.append(alpha*delta_j[j]) #Correcting factor for b_0 (Biases for Input Nodes)
        
    for j in range(num_hidden_layer_nodes):
        for k in range(m):
            w_1[j,k] = w_1[j,k] + w_1_corr[j,k] #Correcting weights of w_1 (Weights of hidden nodes)
            
    for k in range(m):
        b_1[k] = b_1[k] + b_1_corr[k] #Correcting bias of b_1 (Bias of hidden nodes)
    
    for i in range(n):
        for j in range(num_hidden_layer_nodes):
            w_0[i,j] = w_0[i,j] + w_0_corr[i,j] #Correcting weights of w_0 (Weights of input nodes)
            
    for j in range(num_hidden_layer_nodes):
        b_0[j] = b_0[j] + b_0_corr[j]  #Correcting bias of b_0 (Bias of input nodes)

Starting Iteration 0.000000/5000, Training is 0.000000 complete
Starting Iteration 1.000000/5000, Training is 0.020000 complete
Starting Iteration 2.000000/5000, Training is 0.040000 complete
Starting Iteration 3.000000/5000, Training is 0.060000 complete
Starting Iteration 4.000000/5000, Training is 0.080000 complete
Starting Iteration 5.000000/5000, Training is 0.100000 complete
Starting Iteration 6.000000/5000, Training is 0.120000 complete
Starting Iteration 7.000000/5000, Training is 0.140000 complete
Starting Iteration 8.000000/5000, Training is 0.160000 complete
Starting Iteration 9.000000/5000, Training is 0.180000 complete
Starting Iteration 10.000000/5000, Training is 0.200000 complete
Starting Iteration 11.000000/5000, Training is 0.220000 complete
Starting Iteration 12.000000/5000, Training is 0.240000 complete
Starting Iteration 13.000000/5000, Training is 0.260000 complete
Starting Iteration 14.000000/5000, Training is 0.280000 complete
Starting Iteration 15.000000/5000, 

Starting Iteration 175.000000/5000, Training is 3.500000 complete
Starting Iteration 176.000000/5000, Training is 3.520000 complete
Starting Iteration 177.000000/5000, Training is 3.540000 complete
Starting Iteration 178.000000/5000, Training is 3.560000 complete
Starting Iteration 179.000000/5000, Training is 3.580000 complete
Starting Iteration 180.000000/5000, Training is 3.600000 complete
Starting Iteration 181.000000/5000, Training is 3.620000 complete
Starting Iteration 182.000000/5000, Training is 3.640000 complete
Starting Iteration 183.000000/5000, Training is 3.660000 complete
Starting Iteration 184.000000/5000, Training is 3.680000 complete
Starting Iteration 185.000000/5000, Training is 3.700000 complete
Starting Iteration 186.000000/5000, Training is 3.720000 complete
Starting Iteration 187.000000/5000, Training is 3.740000 complete
Starting Iteration 188.000000/5000, Training is 3.760000 complete
Starting Iteration 189.000000/5000, Training is 3.780000 complete
Starting I

Starting Iteration 372.000000/5000, Training is 7.440000 complete
Starting Iteration 373.000000/5000, Training is 7.460000 complete
Starting Iteration 374.000000/5000, Training is 7.480000 complete
Starting Iteration 375.000000/5000, Training is 7.500000 complete
Starting Iteration 376.000000/5000, Training is 7.520000 complete
Starting Iteration 377.000000/5000, Training is 7.540000 complete
Starting Iteration 378.000000/5000, Training is 7.560000 complete
Starting Iteration 379.000000/5000, Training is 7.580000 complete
Starting Iteration 380.000000/5000, Training is 7.600000 complete
Starting Iteration 381.000000/5000, Training is 7.620000 complete
Starting Iteration 382.000000/5000, Training is 7.640000 complete
Starting Iteration 383.000000/5000, Training is 7.660000 complete
Starting Iteration 384.000000/5000, Training is 7.680000 complete
Starting Iteration 385.000000/5000, Training is 7.700000 complete
Starting Iteration 386.000000/5000, Training is 7.720000 complete
Starting I

Starting Iteration 575.000000/5000, Training is 11.500000 complete
Starting Iteration 576.000000/5000, Training is 11.520000 complete
Starting Iteration 577.000000/5000, Training is 11.540000 complete
Starting Iteration 578.000000/5000, Training is 11.560000 complete
Starting Iteration 579.000000/5000, Training is 11.580000 complete
Starting Iteration 580.000000/5000, Training is 11.600000 complete
Starting Iteration 581.000000/5000, Training is 11.620000 complete
Starting Iteration 582.000000/5000, Training is 11.640000 complete
Starting Iteration 583.000000/5000, Training is 11.660000 complete
Starting Iteration 584.000000/5000, Training is 11.680000 complete
Starting Iteration 585.000000/5000, Training is 11.700000 complete
Starting Iteration 586.000000/5000, Training is 11.720000 complete
Starting Iteration 587.000000/5000, Training is 11.740000 complete
Starting Iteration 588.000000/5000, Training is 11.760000 complete
Starting Iteration 589.000000/5000, Training is 11.780000 comp

Starting Iteration 742.000000/5000, Training is 14.840000 complete
Starting Iteration 743.000000/5000, Training is 14.860000 complete
Starting Iteration 744.000000/5000, Training is 14.880000 complete
Starting Iteration 745.000000/5000, Training is 14.900000 complete
Starting Iteration 746.000000/5000, Training is 14.920000 complete
Starting Iteration 747.000000/5000, Training is 14.940000 complete
Starting Iteration 748.000000/5000, Training is 14.960000 complete
Starting Iteration 749.000000/5000, Training is 14.980000 complete
Starting Iteration 750.000000/5000, Training is 15.000000 complete
Starting Iteration 751.000000/5000, Training is 15.020000 complete
Starting Iteration 752.000000/5000, Training is 15.040000 complete
Starting Iteration 753.000000/5000, Training is 15.060000 complete
Starting Iteration 754.000000/5000, Training is 15.080000 complete
Starting Iteration 755.000000/5000, Training is 15.100000 complete
Starting Iteration 756.000000/5000, Training is 15.120000 comp

...

Starting Iteration 4162.000000/5000, Training is 83.240000 complete
Starting Iteration 4163.000000/5000, Training is 83.260000 complete
Starting Iteration 4164.000000/5000, Training is 83.280000 complete
Starting Iteration 4165.000000/5000, Training is 83.300000 complete
Starting Iteration 4166.000000/5000, Training is 83.320000 complete
Starting Iteration 4167.000000/5000, Training is 83.340000 complete
Starting Iteration 4168.000000/5000, Training is 83.360000 complete
Starting Iteration 4169.000000/5000, Training is 83.380000 complete
Starting Iteration 4170.000000/5000, Training is 83.400000 complete
Starting Iteration 4171.000000/5000, Training is 83.420000 complete
Starting Iteration 4172.000000/5000, Training is 83.440000 complete
Starting Iteration 4173.000000/5000, Training is 83.460000 complete
Starting Iteration 4174.000000/5000, Training is 83.480000 complete
Starting Iteration 4175.000000/5000, Training is 83.500000 complete
Starting Iteration 4176.000000/5000, Training is

Starting Iteration 4326.000000/5000, Training is 86.520000 complete
Starting Iteration 4327.000000/5000, Training is 86.540000 complete
Starting Iteration 4328.000000/5000, Training is 86.560000 complete
Starting Iteration 4329.000000/5000, Training is 86.580000 complete
Starting Iteration 4330.000000/5000, Training is 86.600000 complete
Starting Iteration 4331.000000/5000, Training is 86.620000 complete
Starting Iteration 4332.000000/5000, Training is 86.640000 complete
Starting Iteration 4333.000000/5000, Training is 86.660000 complete
Starting Iteration 4334.000000/5000, Training is 86.680000 complete
Starting Iteration 4335.000000/5000, Training is 86.700000 complete
Starting Iteration 4336.000000/5000, Training is 86.720000 complete
Starting Iteration 4337.000000/5000, Training is 86.740000 complete
Starting Iteration 4338.000000/5000, Training is 86.760000 complete
Starting Iteration 4339.000000/5000, Training is 86.780000 complete
Starting Iteration 4340.000000/5000, Training is

Starting Iteration 4511.000000/5000, Training is 90.220000 complete
Starting Iteration 4512.000000/5000, Training is 90.240000 complete
Starting Iteration 4513.000000/5000, Training is 90.260000 complete
Starting Iteration 4514.000000/5000, Training is 90.280000 complete
Starting Iteration 4515.000000/5000, Training is 90.300000 complete
Starting Iteration 4516.000000/5000, Training is 90.320000 complete
Starting Iteration 4517.000000/5000, Training is 90.340000 complete
Starting Iteration 4518.000000/5000, Training is 90.360000 complete
Starting Iteration 4519.000000/5000, Training is 90.380000 complete
Starting Iteration 4520.000000/5000, Training is 90.400000 complete
Starting Iteration 4521.000000/5000, Training is 90.420000 complete
Starting Iteration 4522.000000/5000, Training is 90.440000 complete
Starting Iteration 4523.000000/5000, Training is 90.460000 complete
Starting Iteration 4524.000000/5000, Training is 90.480000 complete
Starting Iteration 4525.000000/5000, Training is

Starting Iteration 4763.000000/5000, Training is 95.260000 complete
Starting Iteration 4764.000000/5000, Training is 95.280000 complete
Starting Iteration 4765.000000/5000, Training is 95.300000 complete
Starting Iteration 4766.000000/5000, Training is 95.320000 complete
Starting Iteration 4767.000000/5000, Training is 95.340000 complete
Starting Iteration 4768.000000/5000, Training is 95.360000 complete
Starting Iteration 4769.000000/5000, Training is 95.380000 complete
Starting Iteration 4770.000000/5000, Training is 95.400000 complete
Starting Iteration 4771.000000/5000, Training is 95.420000 complete
Starting Iteration 4772.000000/5000, Training is 95.440000 complete
Starting Iteration 4773.000000/5000, Training is 95.460000 complete
Starting Iteration 4774.000000/5000, Training is 95.480000 complete
Starting Iteration 4775.000000/5000, Training is 95.500000 complete
Starting Iteration 4776.000000/5000, Training is 95.520000 complete
Starting Iteration 4777.000000/5000, Training is

Starting Iteration 4985.000000/5000, Training is 99.700000 complete
Starting Iteration 4986.000000/5000, Training is 99.720000 complete
Starting Iteration 4987.000000/5000, Training is 99.740000 complete
Starting Iteration 4988.000000/5000, Training is 99.760000 complete
Starting Iteration 4989.000000/5000, Training is 99.780000 complete
Starting Iteration 4990.000000/5000, Training is 99.800000 complete
Starting Iteration 4991.000000/5000, Training is 99.820000 complete
Starting Iteration 4992.000000/5000, Training is 99.840000 complete
Starting Iteration 4993.000000/5000, Training is 99.860000 complete
Starting Iteration 4994.000000/5000, Training is 99.880000 complete
Starting Iteration 4995.000000/5000, Training is 99.900000 complete
Starting Iteration 4996.000000/5000, Training is 99.920000 complete
Starting Iteration 4997.000000/5000, Training is 99.940000 complete
Starting Iteration 4998.000000/5000, Training is 99.960000 complete
Starting Iteration 4999.000000/5000, Training is

In [181]:
def get_pred(Y):
    mx = get_max(Y)
    pred=[]
    for y in Y:
        if mx == y:
            pred.append(1)
        else:
            pred.append(0)
    return pred

In [182]:
print(get_pred(Y))

[0, 0, 1, 0]


In [188]:
results=[]
for x_row in range(x_test.shape[0]):
    
    print("Starting Iteration %s/%s, Training is %s percent complete"%(str(x_row),str(x_test.shape[0]),str(x_row/x_test.shape[0]*100)))
    
    # FEEDFORWARD
    
    H = []
    H_row = []
    Neu_out = 0
    Z_inj = []

    for j in range(num_hidden_layer_nodes):
        Neu_out=0
        for i in range(n):
            Neu_out += x_train[x_row,i] * w_0[i,j] #SUM of x[0]*w_0[0]+x[1]*w_0[1]...
        Neu_out +=b_0[j]
        
        Z_inj.append(Neu_out)
        
        H.append(sigmoid(Neu_out)) #output of hidden layer nodes
                                    #H[j] -> j = Hidden Layer Neuron Index
    
    Y=[]
    Y_row = []
    Neu_out = 0


# # for n_input, weight in zip(x_train[0],w_0):
# #     Neu_out += n_input*weight
#     for k in range(len(H)):
    #for j in range(w_1.shape[0]):
    for k  in range(m):
        Neu_out = 0
        for j in range(num_hidden_layer_nodes):
            Neu_out+=H[j]*w_1[j,k]
        Neu_out +=b_1[k]
        
        Y.append(sigmoid(Neu_out)) #output of output nodes
                                    #Y[k] -> k = Output Layer Neuron index
        
    results.append(get_pred(Y))

Starting Iteration 0/6000, Training is 0.0 percent complete
Starting Iteration 1/6000, Training is 0.016666666666666666 percent complete
Starting Iteration 2/6000, Training is 0.03333333333333333 percent complete
Starting Iteration 3/6000, Training is 0.05 percent complete
Starting Iteration 4/6000, Training is 0.06666666666666667 percent complete
Starting Iteration 5/6000, Training is 0.08333333333333334 percent complete
Starting Iteration 6/6000, Training is 0.1 percent complete
Starting Iteration 7/6000, Training is 0.11666666666666668 percent complete
Starting Iteration 8/6000, Training is 0.13333333333333333 percent complete
Starting Iteration 9/6000, Training is 0.15 percent complete
Starting Iteration 10/6000, Training is 0.16666666666666669 percent complete
Starting Iteration 11/6000, Training is 0.18333333333333332 percent complete
Starting Iteration 12/6000, Training is 0.2 percent complete
Starting Iteration 13/6000, Training is 0.21666666666666665 percent complete
Starting 

Starting Iteration 196/6000, Training is 3.266666666666666 percent complete
Starting Iteration 197/6000, Training is 3.283333333333333 percent complete
Starting Iteration 198/6000, Training is 3.3000000000000003 percent complete
Starting Iteration 199/6000, Training is 3.3166666666666664 percent complete
Starting Iteration 200/6000, Training is 3.3333333333333335 percent complete
Starting Iteration 201/6000, Training is 3.35 percent complete
Starting Iteration 202/6000, Training is 3.3666666666666663 percent complete
Starting Iteration 203/6000, Training is 3.3833333333333333 percent complete
Starting Iteration 204/6000, Training is 3.4000000000000004 percent complete
Starting Iteration 205/6000, Training is 3.4166666666666665 percent complete
Starting Iteration 206/6000, Training is 3.4333333333333336 percent complete
Starting Iteration 207/6000, Training is 3.45 percent complete
Starting Iteration 208/6000, Training is 3.4666666666666663 percent complete
Starting Iteration 209/6000, 

Starting Iteration 508/6000, Training is 8.466666666666667 percent complete
Starting Iteration 509/6000, Training is 8.483333333333333 percent complete
Starting Iteration 510/6000, Training is 8.5 percent complete
Starting Iteration 511/6000, Training is 8.516666666666667 percent complete
Starting Iteration 512/6000, Training is 8.533333333333333 percent complete
Starting Iteration 513/6000, Training is 8.55 percent complete
Starting Iteration 514/6000, Training is 8.566666666666666 percent complete
Starting Iteration 515/6000, Training is 8.583333333333334 percent complete
Starting Iteration 516/6000, Training is 8.6 percent complete
Starting Iteration 517/6000, Training is 8.616666666666667 percent complete
Starting Iteration 518/6000, Training is 8.633333333333333 percent complete
Starting Iteration 519/6000, Training is 8.649999999999999 percent complete
Starting Iteration 520/6000, Training is 8.666666666666668 percent complete
Starting Iteration 521/6000, Training is 8.6833333333

Starting Iteration 959/6000, Training is 15.983333333333333 percent complete
Starting Iteration 960/6000, Training is 16.0 percent complete
Starting Iteration 961/6000, Training is 16.01666666666667 percent complete
Starting Iteration 962/6000, Training is 16.03333333333333 percent complete
Starting Iteration 963/6000, Training is 16.05 percent complete
Starting Iteration 964/6000, Training is 16.066666666666666 percent complete
Starting Iteration 965/6000, Training is 16.083333333333332 percent complete
Starting Iteration 966/6000, Training is 16.1 percent complete
Starting Iteration 967/6000, Training is 16.116666666666667 percent complete
Starting Iteration 968/6000, Training is 16.133333333333333 percent complete
Starting Iteration 969/6000, Training is 16.150000000000002 percent complete
Starting Iteration 970/6000, Training is 16.166666666666664 percent complete
Starting Iteration 971/6000, Training is 16.183333333333334 percent complete
Starting Iteration 972/6000, Training is 1

Starting Iteration 1178/6000, Training is 19.633333333333333 percent complete
Starting Iteration 1179/6000, Training is 19.650000000000002 percent complete
Starting Iteration 1180/6000, Training is 19.666666666666664 percent complete
Starting Iteration 1181/6000, Training is 19.683333333333334 percent complete
Starting Iteration 1182/6000, Training is 19.7 percent complete
Starting Iteration 1183/6000, Training is 19.716666666666665 percent complete
Starting Iteration 1184/6000, Training is 19.733333333333334 percent complete
Starting Iteration 1185/6000, Training is 19.75 percent complete
Starting Iteration 1186/6000, Training is 19.766666666666666 percent complete
Starting Iteration 1187/6000, Training is 19.783333333333335 percent complete
Starting Iteration 1188/6000, Training is 19.8 percent complete
Starting Iteration 1189/6000, Training is 19.816666666666666 percent complete
Starting Iteration 1190/6000, Training is 19.833333333333332 percent complete
Starting Iteration 1191/600

...

Starting Iteration 5240/6000, Training is 87.33333333333333 percent complete
Starting Iteration 5241/6000, Training is 87.35000000000001 percent complete
Starting Iteration 5242/6000, Training is 87.36666666666667 percent complete
Starting Iteration 5243/6000, Training is 87.38333333333334 percent complete
Starting Iteration 5244/6000, Training is 87.4 percent complete
Starting Iteration 5245/6000, Training is 87.41666666666667 percent complete
Starting Iteration 5246/6000, Training is 87.43333333333332 percent complete
Starting Iteration 5247/6000, Training is 87.45 percent complete
Starting Iteration 5248/6000, Training is 87.46666666666667 percent complete
Starting Iteration 5249/6000, Training is 87.48333333333333 percent complete
Starting Iteration 5250/6000, Training is 87.5 percent complete
Starting Iteration 5251/6000, Training is 87.51666666666667 percent complete
Starting Iteration 5252/6000, Training is 87.53333333333333 percent complete
Starting Iteration 5253/6000, Trainin

Starting Iteration 5471/6000, Training is 91.18333333333334 percent complete
Starting Iteration 5472/6000, Training is 91.2 percent complete
Starting Iteration 5473/6000, Training is 91.21666666666667 percent complete
Starting Iteration 5474/6000, Training is 91.23333333333333 percent complete
Starting Iteration 5475/6000, Training is 91.25 percent complete
Starting Iteration 5476/6000, Training is 91.26666666666667 percent complete
Starting Iteration 5477/6000, Training is 91.28333333333333 percent complete
Starting Iteration 5478/6000, Training is 91.3 percent complete
Starting Iteration 5479/6000, Training is 91.31666666666666 percent complete
Starting Iteration 5480/6000, Training is 91.33333333333333 percent complete
Starting Iteration 5481/6000, Training is 91.35 percent complete
Starting Iteration 5482/6000, Training is 91.36666666666666 percent complete
Starting Iteration 5483/6000, Training is 91.38333333333334 percent complete
Starting Iteration 5484/6000, Training is 91.4 pe

Starting Iteration 5688/6000, Training is 94.8 percent complete
Starting Iteration 5689/6000, Training is 94.81666666666668 percent complete
Starting Iteration 5690/6000, Training is 94.83333333333334 percent complete
Starting Iteration 5691/6000, Training is 94.85 percent complete
Starting Iteration 5692/6000, Training is 94.86666666666666 percent complete
Starting Iteration 5693/6000, Training is 94.88333333333333 percent complete
Starting Iteration 5694/6000, Training is 94.89999999999999 percent complete
Starting Iteration 5695/6000, Training is 94.91666666666667 percent complete
Starting Iteration 5696/6000, Training is 94.93333333333334 percent complete
Starting Iteration 5697/6000, Training is 94.95 percent complete
Starting Iteration 5698/6000, Training is 94.96666666666667 percent complete
Starting Iteration 5699/6000, Training is 94.98333333333333 percent complete
Starting Iteration 5700/6000, Training is 95.0 percent complete
Starting Iteration 5701/6000, Training is 95.0166

Starting Iteration 5831/6000, Training is 97.18333333333334 percent complete
Starting Iteration 5832/6000, Training is 97.2 percent complete
Starting Iteration 5833/6000, Training is 97.21666666666667 percent complete
Starting Iteration 5834/6000, Training is 97.23333333333333 percent complete
Starting Iteration 5835/6000, Training is 97.25 percent complete
Starting Iteration 5836/6000, Training is 97.26666666666667 percent complete
Starting Iteration 5837/6000, Training is 97.28333333333333 percent complete
Starting Iteration 5838/6000, Training is 97.3 percent complete
Starting Iteration 5839/6000, Training is 97.31666666666666 percent complete
Starting Iteration 5840/6000, Training is 97.33333333333334 percent complete
Starting Iteration 5841/6000, Training is 97.35000000000001 percent complete
Starting Iteration 5842/6000, Training is 97.36666666666667 percent complete
Starting Iteration 5843/6000, Training is 97.38333333333333 percent complete
Starting Iteration 5844/6000, Trainin

Starting Iteration 5979/6000, Training is 99.65 percent complete
Starting Iteration 5980/6000, Training is 99.66666666666667 percent complete
Starting Iteration 5981/6000, Training is 99.68333333333334 percent complete
Starting Iteration 5982/6000, Training is 99.7 percent complete
Starting Iteration 5983/6000, Training is 99.71666666666667 percent complete
Starting Iteration 5984/6000, Training is 99.73333333333333 percent complete
Starting Iteration 5985/6000, Training is 99.75 percent complete
Starting Iteration 5986/6000, Training is 99.76666666666667 percent complete
Starting Iteration 5987/6000, Training is 99.78333333333333 percent complete
Starting Iteration 5988/6000, Training is 99.8 percent complete
Starting Iteration 5989/6000, Training is 99.81666666666666 percent complete
Starting Iteration 5990/6000, Training is 99.83333333333333 percent complete
Starting Iteration 5991/6000, Training is 99.85000000000001 percent complete
Starting Iteration 5992/6000, Training is 99.8666

In [189]:
print(results)

[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0

In [190]:
ground_truth = []
for i in range(len(Sev_none_test)):
    ground_truth.append([Sev_none_test[i],Sev_mild_test[i],Sev_mod_test[i],Sev_sev_test[i]])

In [191]:
count=0
for i in range(len(ground_truth)):
    if ground_truth[i]==results[i]:
        count+=1

In [192]:
print(count/len(ground_truth))

0.25433333333333336
